In [69]:
# Import libraries
import io
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.neural_network import MLPRegressor


# Load data 
url = 'https://drive.switch.ch/index.php/s/37RuoA3Mgt9Rqah/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))


# Alternatively you can load the data from file
#data_path = 'data.npz' # path to the .npz file storing the data
#data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# T1 (Linear Regression)
# regularize data
pol_feat_high = PolynomialFeatures(degree=2, include_bias=False)
X = pol_feat_high.fit_transform(x)

# augment so it fits specified model
# X = np.hstack((x, 
#     np.cos(x[:,0]).reshape(-1,1),
#     (x[:,1]**2).reshape(-1,1),
#     (np.tanh(x[:,0]).reshape(-1,1))
# ))

# train/test split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, shuffle=True, random_state=0)

# modify and reshape train/test input to fit our family of models
x_train_aug = np.hstack((x_train, 
    np.cos(x_train[:,0]).reshape(-1,1),
    (x_train[:,1]**2).reshape(-1,1),
    (np.tanh(x_train[:,0]).reshape(-1,1))
))

x_test_aug = np.hstack((x_test, 
    np.cos(x_test[:,0]).reshape(-1,1),
    (x_test[:,1]**2).reshape(-1,1),
    (np.tanh(x_test[:,0]).reshape(-1,1))
))

# regression
lr = LinearRegression()
lr.fit(x_train_aug, y_train)

theta = [lr.intercept_] + lr.coef_.tolist()
print('theta = {}\n'.format(theta))

# mean squared error / performance measure
train_pred = lr.predict(x_train_aug)
test_pred = lr.predict(x_test_aug)

mse_train = mean_squared_error(y_train, train_pred)
print("Mean squared error (train): {}".format(mse_train))

mse_test = mean_squared_error(y_test, test_pred)
print("Mean squared error (test): {}\n".format(mse_test))

# ========================= Lasso Regression ============================
interval = np.linspace(-1, 2, 1000).reshape(1000,1)
scaler = StandardScaler()
sigma = 0.02;
lasso = Lasso(sigma)
lasso.fit(x_train_aug, y_train)

lasso_train_pred = lasso.predict(x_train_aug)
lasso_test_pred = lasso.predict(x_test_aug)

lasso_mse_train = mean_squared_error(y_train, lasso_train_pred)
print("Mean squared error (Lasso train): {}".format(lasso_mse_train))

lasso_mse_test = mean_squared_error(y_test, lasso_test_pred)
print("Mean squared error (Lasso test): {}\n".format(lasso_mse_test))
# ========================================================================

# T2 (Non-Linear Regression)
# regularize data
pol_feat_high = PolynomialFeatures(degree=5, include_bias=False)
X = pol_feat_high.fit_transform(x)

nn_kwargs= {"hidden_layer_sizes":(200,), # number of neurons
            "activation":"relu",         # non-linear activation function (through all the network)
            "max_iter":250,              # epochs
            "solver":"adam",             # optimizer
            "random_state": 42}           

ffnn = MLPRegressor(**nn_kwargs)

X = np.vstack(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=0)

ffnn.fit(X, y)

ffnn_train_pred = ffnn.predict(X_train)
ffnn_test_pred = ffnn.predict(X_test)

ffnn_mse_train = mean_squared_error(Y_train, ffnn_train_pred)
print("Mean squared error (FFNN train): {}".format(ffnn_mse_train))

ffnn_mse_test = mean_squared_error(Y_test, ffnn_test_pred)
print("Mean squared error (FFNN test): {}".format(ffnn_mse_test))

# T3 (Bonus)
# print("Everything useful\n")

theta = [0.9655139152197769, 5.051538138473209, -4.004896218606084, 7.021097556897876, 1.9976899691291532, -0.10373532944839803]

Mean squared error (train): 1.4136786666328538
Mean squared error (test): 1.4921846102530003

Mean squared error (Lasso train): 1.4171483544297718
Mean squared error (Lasso test): 1.4877102711352492

Mean squared error (FFNN train): 2.604005820747914
Mean squared error (FFNN test): 2.6816303389671337


In [10]:
import joblib
# Import libraries
import io
import requests
import torch
import numpy as np

def evaluate_predictions(y_true, y_pred):
    """
    Evaluates the mean squared error between the values in y_true and the values
    in y_pred.
    ### YOU CAN NOT EDIT THIS FUNCTION ###
    :param y_true: Numpy array, the true target values from the test set;
    :param y_pred: Numpy array, the values predicted by your model.
    :return: float, the mean squared error between the two arrays.
    """
    assert y_true.shape == y_pred.shape
    return ((y_true - y_pred) ** 2).mean()


def load_model(filename):
    """
    Loads a torch model saved.
    This is just an example, you can write your own function to load the model.
    Some examples can be found in src/utils.py.
    :param filename: string, path to the file storing the model.
    :return: the model.
    """
    model = torch.jit.load(filename)

    return model

# Load the data
# This will be replaced with our private test data when grading the assignment

# Load data from url
url = 'https://drive.switch.ch/index.php/s/Wp0I2gb33mhERFN/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# Alternatively yo can load the data from file
#data_path = 'data_bonus_test.npz'
#data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = torch.tensor(data.f.x, dtype=torch.float32)
# y is a Numpy array of shape (n_samples, ) with the targets
y =  torch.tensor(data.f.y,dtype=torch.float32).reshape(-1, 1)

# Load the trained model
baseline_model_path = r'Torch Model/baseline.pt'
baseline_model = load_model(baseline_model_path)

# Predict on the given samples
y_pred_ours = baseline_model(x)

############################################################################
# STOP EDITABLE SECTION: DO NOT modify anything above this point.
############################################################################

############################################################################
# ADD HERE YOUR CODE TO READ MODEL OF TASK 3
############################################################################




# Load the trained model
baseline_model_path = 'YOUR_MODEL_PATH'
baseline_model =  ...   # LOAD YOU MODEL and predict x
# Predict on the given samples FROM YOUR MODEL
y_pred_yours = ...


############################################################################
# STOP EDITABLE SECTION: do not modify anything below this point.
############################################################################

# Evaluate the prediction using MSE
mse = evaluate_predictions(y_pred_yours, y)
print(f'MSE on whole dataset: {mse}')

# NOTE: NOW THIS CELL IS NOT WORKING SINCE YOU NEED TO CHANGE THE INPUT.
# DO IT AND EVERYTHING RUNS SMOOTH

AttributeError: 'ellipsis' object has no attribute 'shape'